In [ ]:
import json

deletions = json.load(open('Confussion Sets/confussion_set_deletion.json', 'r', encoding='utf-8'))
insertions = json.load(open('Confussion Sets/confussion_set_insertion.json', 'r', encoding='utf-8'))
substitutions = json.load(open('Confussion Sets/confussion_set_substitution.json', 'r', encoding='utf-8'))

In [ ]:
#no of elements in each error list
x=10

In [ ]:
from collections import defaultdict
from unicodedata import normalize as unicodeNormalize
import regex as re

suffixesList = [ 
    "්", 
    "ා", 
    "ැ", 
    "ෑ", 
    "ි", 
    "ී", 
    "ු", 
    "ූ", 
    "ෙ", 
    "ේ", 
    "ෛ", 
    "ො", 
    "ෝ" ,
    "ෞ",
    "ෘ",
    "ෲ"
    ]

def tokenize(text):
    tokens = []
    li = 1
    while li < len(text):
        if text[li]=='s':
            tokens.append('<s>')
            li+=2
            continue
    # for li in range(1, len(text)):
        prevChar, currentChar = text[li - 1], text[li]
        if(currentChar == '\u200d'):
                if(li < len(text) - 1):
                    if(prevChar == suffixesList[0] and (text[li + 1] == 'ර' or text[li + 1] == 'ය' or text[li + 1] == 'ද')):
                        tokens.append(tokens.pop()+currentChar+text[li + 1])
                        li += 1
        elif(currentChar in suffixesList):
            if(li != 1):
                tokens.append(f"{tokens.pop()}{currentChar}")
            else:
                tokens.append(f"{prevChar}{currentChar}")
        else:
            if(li == 1):
                tokens.append(prevChar)
            tokens.append(currentChar)
        li += 1
    return tokens

In [ ]:
lines = []
with open('c:\\Users\\Yasith\\Documents\\GitHub\\FYP-Omega\\Data/combined all names - cleaned - train.txt', 'r', encoding='utf-8') as f:
    allCharacters = []
    for line in f.readlines():
        line = re.sub(r'[^\w]|_|[a-zA-Z]|\u200c', ' ', line)
        line = unicodeNormalize("NFC", line.strip())
        for sub in line.split():
            sub = tokenize(sub)
            if(len(sub) > 1):
                lines.append(sub)
            allCharacters = list(set(allCharacters + sub))

In [ ]:
unigramCount = defaultdict(int)
for line in lines:
    for ch in line:
        unigramCount[ch] += 1
unigramCount['<s>']=len(lines)


In [ ]:
bigramCount = defaultdict(int)
for name in lines:
    biName=name[:]
    biName.insert(0,'<s>')
    for i in range(len(biName)-1):
        bigramCount[(biName[i],biName[i+1])] += 1

In [ ]:
bigramCount

In [ ]:
deleteProbs = defaultdict()
for key,value in deletions.items():
    if key.split(',')[1]=='-':  #multi errors. improve
        continue
    tokens = tokenize(key.split(',')[0])
    if len(tokens)<2:
        print(tokens)
        continue
    if tokens[0] not in deleteProbs:
        deleteProbs[tokens[0]] = []
    prob = (value+1)/(bigramCount[tuple(tokens)]+1)
    deleteProbs[tokens[0]] .append((tokens[1],prob))
    #print (tokenize(key.split(',')[0]),value
for key,value in deleteProbs.items():
    if len(value)>10:
        deleteProbs[key] = sorted(value, key=lambda tup: tup[1], reverse=True)[:x]
deleteProbs

In [ ]:
insertProbs = defaultdict()
for key,value in insertions.items():
    if key.split(',')[0]=='-':  #multi errors. improve
        continue
    tokens = tokenize(key.split(',')[1])
    if len(tokens)<2:
        print(key,tokens)
        continue
    if tokens[1] not in insertProbs:
        insertProbs[tokens[1]] = []
    prob = (value+1)/(unigramCount[tokens[0]]+1)
    insertProbs[tokens[1]].append((tokens[0],prob))
    #print (tokenize(key.split(',')[0]),value
for key,value in insertProbs.items():
    if len(value)>10:
        insertProbs[key] = sorted(value, key=lambda tup: tup[1], reverse=True)[:x]
insertProbs

In [45]:
substitutions

{'සෙ,බෙ': 30,
 'ල්,ලේ': 358,
 'තු,නු': 987,
 'ත,න': 6614,
 'ති,නි': 12970,
 'ම්,මි': 1775,
 'ල,න': 327,
 'තා,නා': 2194,
 'ඩු,ඩුූ': 17,
 'ලා,ළා': 718,
 'මි,මීි': 98,
 'හ,ං': 6091,
 'ච,එ': 945,
 'තු,ත': 1743,
 'පී,පිී': 38,
 'රි,ර': 17661,
 'වි,ප': 251,
 'ෂා,බා': 103,
 'ර,ර්': 847,
 'හේ,භේ': 141,
 'ර,එ': 1017,
 'ක්\u200dරි,ඩ': 1,
 'ස්,ට්\u200dර්': 1,
 'ට්\u200dරි,ය': 1,
 'ට්\u200dරි,ස්්\u200dර්\u200dරි': 1,
 'ද,බැ': 7,
 'ල,ලු': 774,
 'බි,බ': 277,
 'බි,බ්': 1453,
 'ව,ච': 14337,
 'වි,චි': 18358,
 'ජේ,පේ': 6164,
 'ච,ව': 646,
 'ද්\u200dර,දු': 3159,
 'රි,ර්': 16477,
 'ද්\u200dර,ද': 5870,
 'රි,දි': 1237,
 'ඉ,ම': 3153,
 'මා,මා්': 332,
 'කු,ක': 1138,
 'ර්,රී': 95,
 'ෆී,ර්': 5,
 'ත්,බ': 12,
 'ති,බ්': 672,
 'ති,බි': 659,
 'ත,බ': 587,
 'නු,තු': 1239,
 'ත්,න්': 6738,
 'ෂි,ෂ්': 1484,
 'ෂි,ෂ': 2332,
 'ඉ,ප': 13,
 'ගේ,ගේ්': 102,
 'ගේ,ග්': 1199,
 'න්,නේ': 760,
 'හ,න': 9077,
 'මී,මි': 1234,
 'මි,ම්': 19952,
 'ෆ්\u200dරි,ර්\u200dරි': 1,
 'ලු,ල': 416,
 'ඩ්,ඩ': 141,
 'ලු,ලූ': 5336,
 'ර්,ර': 3902,
 'මී,ම': 92

In [46]:
subsProbs = defaultdict()
for key,value in substitutions.items():
    correct, incorrect = key.split(',')
    if incorrect not in subsProbs:
        subsProbs[incorrect] = []
    prob = (value+1)/(unigramCount[correct]+1)
    subsProbs[incorrect].append((correct,prob))
    #print (tokenize(key.split(',')[0]),value
for key,value in subsProbs.items():
    if len(value)>10:
        subsProbs[key] = sorted(value, key=lambda tup: tup[1], reverse=True)[:x]
subsProbs

defaultdict(None,
            {'බෙ': [('බේ', 0.020752797558494403),
              ('කෙ', 0.006321112515802781),
              ('ටෙ', 0.005154639175257732),
              ('සෙ', 0.0035052012663952963),
              ('තෙ', 0.0025399129172714078),
              ('ඔ', 0.0015151515151515152),
              ('කී', 0.0010922992900054614),
              ('බෝ', 0.0009689922480620155),
              ('රො', 0.0009237875288683603),
              ('හෙ', 0.0005276116778051355)],
             'ලේ': [('ෆේ', 0.36363636363636365),
              ('ඹේ', 0.020833333333333332),
              ('ල්', 0.0055562434222745),
              ('ඕ', 0.0024271844660194173),
              ('ෆ්', 0.0021551724137931034),
              ('ෂෙ', 0.002070393374741201),
              ('යේ', 0.0018264840182648401),
              ('ලී', 0.001822231220894918),
              ('පේ', 0.0012669683257918551),
              ('ජේ', 0.0011305112423062428)],
             'නු': [('නූ', 0.43610013175230566),
              ('භු', 0.2),
     

In [47]:

with open('insertion_probabilities.json', 'w', encoding='utf-8') as outfile:
    json.dump(insertProbs, outfile, ensure_ascii=False)

with open('deletion_probabilities.json', 'w', encoding='utf-8') as outfile:
    json.dump(deleteProbs, outfile, ensure_ascii=False)

with open('substitution_probabilities.json', 'w', encoding='utf-8') as outfile:
    json.dump(subsProbs, outfile, ensure_ascii=False)